# Hypothesis Testing 101

This notebook is split into subsections detailing situations for different hypothesis Tests.
Hypothesis testing normally is done on proportion and mean.
- 1.Hypothesis Testing for One Proportion
- 2.Hypothesis Tests for the Difference in Two Proportions
- 3.Hypothesis Testing for One Mean
- 4.Hypothesis Testing for the Difference in Mean
- 5.Chi Squared
- 6.Anova

In [216]:
import numpy as np
import pandas as pd
from scipy import stats

### 2. Hypothesis Tests for the Difference in Two Proportions

Conditions for Inference:  
1) Random Sample (Both  samples are random)    
2) np1 > 10 n(1-p1) > 10 np2 > 10 n(1-p2) > 10  
3) Observations in sample are independant (Either done with replacement or less than 10% of population)

Do not need 30 condition when dealing with proportions, we do for means

#### Example 2A

Duncan is investigating if residents in the city support the construction of a new high school. He is curious avout the differenve of opinion between the noth and the south parts of the city.He obtained the following information as a random sample.


|Supports|North|South|
|---|---|---|
Yes|54|77|
No|66|63|
Total|120|140|

What is the 90% CI in the difference of opinions who support the project (Ps-Pn)


In [181]:
# Confidence Interval between 2 means
p1 = 176/200
p2 = 168/200
p = (176+168)/(400)
n1 = 200
n2 = 200
p21 = p2-p1
conf = 0.99
tail = 2
if tail == 2:
    q = (1+conf)/2
else:
    q= conf


var1_est = p1*(1-p1)/n1
var2_est = p2*(1-p2)/n2

sd_est = np.sqrt(var1_est + var2_est)

z = stats.norm.ppf(q)
error = z*(sd_est)
CI = p21-error, p21+error
# print(p1,p2, p21, var1_est,var2_est,sd_est)
# print(stats.norm.ppf(q))

CI
error


0.08922934450742903

Noe that we are doing a hypothesis test where we are testing for difference in proportions = 0
We use the combined as the standard error of estimate
SE = sqrt(p(1-p) (1/n1 + 1/n2))


In [182]:
# P values on the same data 
# H0 p2-p1 = 0
# H0 p2-p1 ^=0
# Significance = 5%

alpha = 0.05

SE = np.sqrt(p*(1-p)*(1/n1 + 1/n2))
SE

critical_z = p21/SE

critical_z
p_val = stats.norm.cdf(critical_z)
p_val = tail*stats.norm.cdf(-np.abs(critical_z))
p_val
if p_val <= alpha:
    print("We reject the Null Hypothesis with a pvalue {:2f}".format(p_val))
else:
     print("We fail to reject the Null Hypothesis with a pvalue {:2f}".format(p_val))

We fail to reject the Null Hypothesis with a pvalue 0.249000


In [183]:
# P values on the same data 
# H0 p2-p1 = 0
# H0 p2-p1 ^=0
# Significance = 5%

alpha = 0.01

p1 = 168
p2 = 176
p = 0.55
n1 = 200
n2 = 200
p21 = p2-p1

SE = np.sqrt(p*(1-p)*(1/n1 + 1/n2))
SE

critical_z = p21/SE

critical_z
p_val = tail*stats.norm.cdf(-np.abs(critical_z))
p_val
if p_val <= alpha:
    print("We reject the Null Hypothesis with a pvalue {:2f}".format(p_val))
else:
     print("We fail to reject the Null Hypothesis with a pvalue {:2f}".format(p_val))
critical_z

We reject the Null Hypothesis with a pvalue 0.000000


160.80605044147393

#### Convert the above into a function

In [184]:
def compare_two_propotions(s1,s2,n1,n2,t, alpha):
    
    p1 = s1/n1
    p2= s2/n2
    p21 = p2-p1
    p = (s1 + s2)/(n1 +n2)
    SE = np.sqrt(p*(1-p)*(1/n1 + 1/n2))
    critical_z = p21/SE
    p_val = t*stats.norm.cdf(-np.abs(critical_z))
    if p_val <= alpha:
        print("We reject the Null Hypothesis with a pvalue {:2f}".format(p_val))
    else:
        print("We fail to reject the Null Hypothesis with a pvalue {:2f}".format(p_val))
    
    print ("The Z Score is ", critical_z)
    return 

In [185]:
compare_two_propotions(54,77,120,140,2, 0.05)

We fail to reject the Null Hypothesis with a pvalue 0.107896
The Z Score is  1.607721471364662


In [186]:
compare_two_propotions(58,52,100,100,2, 0.05)

We fail to reject the Null Hypothesis with a pvalue 0.393769
The Z Score is  -0.8528028654224409


In [187]:
compare_two_propotions(132,228,400,600,1, 0.05)

We fail to reject the Null Hypothesis with a pvalue 0.053292
The Z Score is  1.6137430609197567


In [188]:
compare_two_propotions(14,24,241,259,1, 0.1)

We reject the Null Hypothesis with a pvalue 0.072463
The Z Score is  1.457690263955363


In [189]:
compare_two_propotions(35,90,50,150,2, 0.05)

We fail to reject the Null Hypothesis with a pvalue 0.205903
The Z Score is  -1.2649110640673513


In [190]:
compare_two_propotions(168,176,200,200,2,0.01)

We fail to reject the Null Hypothesis with a pvalue 0.249000
The Z Score is  1.152780835408471


In [191]:
stats.norm.ppf(.95)

1.6448536269514722

### 2. Hypothesis Tests for the Difference in Two Means

Conditions for Inference:  
1) Random Sample (Both  samples are random)    
2) Normal n>= 30 or if nort underlying dist is approx symmetrical and not skewed  
3) Observations in sample are independant (Either done with replacement or less than 10% of population)

Do not need 30 condition when dealing with proportions, we do for means


When comparing means as opposed to proportion we do not know the population variances so instead we estimate them using the sample populations standard deviation. So we estimate the sd of the sd of the difference in means using  sqrt(s1^2/n1 + s2^2/n2) However once we use this estimate a critical z value is not as good as a critical t value.

So the formula 
(X1- X2) +- t* sqrt(s1^2/n1 + s2^2/n2) 
degrees of freedom  = min(n1,n2) -1

#### Example 3A Confidence Intervals

Kylie suspects that when people exercide longer their body temps change. She finds the following  
18 people who exercised for 30 mins had an average body temp of 38.3 (sd 0.27)  
24 people who exercised for 60 mins had an average body temp of 38.9 (sd 0.29)

We wish to calculate the 90% CI for the difference in mean body temp

In [192]:
x1 = 38.3
n1 = 18
sd1 = 0.27

x2 = 38.9
n2 = 24
sd2 = 0.29
tail = 2
conf =0.9
se = np.sqrt(sd1**2/n1 + sd2**2/n2)
df = min(n1,n2) -1

if tail == 2:
    q = (1+conf)/2
else:
    q= conf

t = stats.t.ppf(q,df)
CI = (x2-x1)-t*se, (x2-x1)+t*se

CI

(0.44880258736742723, 0.7511974126325756)

### Hypothesis Test

In [193]:
def compare_two_means(x1,x2,n1,n2,s1,s2,t, alpha):
    

    x21 = x2-x1
    SE = np.sqrt((s1**2/n1 + s2**2/n2))
    critical_t = x21/SE
    #df = min(n1,n1) -1
    df = (sd1**2/n1 + sd2**2/n2)**2/ ((1/(n1-1)*(sd1**2/n1)**2 + (1/(n2-1)*(sd2**2/n2)**2)))
    p_val = t*stats.t.cdf(-np.abs(critical_t), df)
    if p_val <= alpha:
        print("We reject the Null Hypothesis with a pvalue {:2f}".format(p_val))
    else:
        print("We fail to reject the Null Hypothesis with a pvalue {:2f}".format(p_val))
    
    print ("The t Score is ", critical_t)

    return 

In [194]:
compare_two_means(1.3,1.6,22,24,0.5,0.3,2,.05)

We reject the Null Hypothesis with a pvalue 0.018774
The t Score is  2.440263759933568


In [280]:
compare_two_means(168,172,5,5,5.4,7.5,2,.05)

We fail to reject the Null Hypothesis with a pvalue 0.361622
The t Score is  0.9678111752325689


In [195]:
compare_two_means(116,120,65,65,13,15,2,.01)

We fail to reject the Null Hypothesis with a pvalue 0.106703
The t Score is  1.6246827101404884


In [196]:
def ci_two_means(x1,x2,n1,n2,sd1,sd2,tail,conf):


    se = np.sqrt((sd1**2/n1) + (sd2**2/n2))
   #df = min(n1,n2) -1
    
    df = (sd1**2/n1 + sd2**2/n2)**2/ ((1/(n1-1)*(sd1**2/n1)**2 + (1/(n2-1)*(sd2**2/n2)**2)))
                                                                 
    print("df",df)

    if tail == 2:
        q = (1+conf)/2
    else:
        q= conf

    t = stats.t.ppf(q,df)
    CI = round((x2-x1)-t*se,2), round((x2-x1)+t*se,2)
    print(se,t, se*t)
    return CI


In [197]:
# ci_two_means(120,116,65,65,15,13,2,.99)
ci_two_means(116,120,65,65,13,15,2,.99)

df 125.46541055237036
2.462019183828278 2.615583217219186 6.4396160576929224


(-2.44, 10.44)

In [198]:
ci_two_means(115.3,116.8,9,24,26.2,15.8,2,.90)

df 10.264429114891628
9.309821576044182 1.8077347222982187 16.829687721416196


(-15.33, 18.33)

In [199]:
ci_two_means(11700,9629,6,7,4900,3200,2,.95)

df 8.389516095670354
2337.6320945614625 2.287500236945164 5347.333970199966


(-7418.33, 3276.33)

In [200]:
compare_two_means(10.8,10.3,46,58,2.3,2.5,1,.01)

We fail to reject the Null Hypothesis with a pvalue 0.145997
The t Score is  -1.0593823774595397


### Power

The test will calculate a p-value that can be interpreted as to whether the samples are the same (fail to reject the null hypothesis), or there is a statistically significant difference between the samples (reject the null hypothesis). A common significance level for interpreting the p-value is 5% or 0.05.

**Significance level (alpha)**: 5% or 0.05.
The size of the effect of comparing two groups can be quantified with an effect size measure. A common measure for comparing the difference in the mean from two groups is the Cohen’s d measure. It calculates a standard score that describes the difference in terms of the number of standard deviations that the means are different. A large effect size for Cohen’s d is 0.80 or higher, as is commonly accepted when using the measure.

**Effect Size**: Cohen’s d of at least 0.80.
We can use the default and assume a minimum statistical power of 80% or 0.8.

**Statistical Power**: 80% or 0.80.
For a given experiment with these defaults, we may be interested in estimating a suitable sample size. That is, how many observations are required from each sample in order to at least detect an effect of 0.80 with an 80% chance of detecting the effect if it is true (20% of a Type II error) and a 5% chance of detecting an effect if there is no such effect (Type I error).

We can solve this using a power analysis.



A note on sample size: the function has an argument called ratio that is the ratio of the number of samples in one sample to the other. If both samples are expected to have the same number of observations, then the ratio is 1.0. If, for example, the second sample is expected to have half as many observations, then the ratio would be 0.5.

In [201]:
from statsmodels.stats.power import TTestIndPower

effect_size = 0.5
alpha = 0.05
power = 0.8
# perform power analysis
analysis = TTestIndPower()
sample_size = analysis.solve_power(effect_size, power=power, nobs1=None, ratio=1.0, alpha=alpha)
print('Sample Size: %.3f' % sample_size)

Sample Size: 63.766


In [202]:
(82-80)/9

0.2222222222222222

In [203]:
.05*.95 

0.0475

## 5 Chi Squared - Goodness of fit  How well does our observations fot against the expected values

The chi-square goodness of fit test is appropriate when the following conditions are met: The sampling method is simple random sampling. The variable under study is categorical. The expected value ( Note expected NOT Observed) of the number of sample observations in each level of the variable is at least 5.

In [235]:
observed = np.array([75, 55, 15, 5])
expected_ratio =  np.array([0.5, 0.3, 0.1, 0.1])
expected = observed.sum()*expected_ratio
stats.chisquare(observed, expected)

Power_divergenceResult(statistic=8.88888888888889, pvalue=0.03080523378048964)

Joe wondered whether the days of the week of health clinic appointments at his clinic had an even distribution from Monday through Friday, so he took a random sample of 500health clinic appointments and recorded their days of the week. Here are his results:  
M 115  
T 100  
W 115  
T 100  
F 70  

In [252]:
observed = np.array([115, 100, 115, 100, 70])
expected_ratio =  np.array([0.2, 0.2, 0.2, 0.2, 0.2])
expected = observed.sum()*expected_ratio
stats.chisquare(observed, expected)



Power_divergenceResult(statistic=13.5, pvalue=0.009074317061131603)

We reject the null hypothesis that observed = expected as p value is below our significance alpha of 0.05

In [240]:
observed = np.array([345,125,30])
expected_ratio =  np.array([0.66, 0.25, 0.09])
expected = observed.sum()*expected_ratio
stats.chisquare(observed, expected)

Power_divergenceResult(statistic=5.681818181818182, pvalue=0.05837257585823774)

In [241]:
observed = np.array([75,120,90,15])
expected_ratio =  np.array([0.25,0.40,  0.25, 0.10])
expected = observed.sum()*expected_ratio
stats.chisquare(observed, expected)

Power_divergenceResult(statistic=10.5, pvalue=0.014760897143990665)

In [276]:
observed = np.array([12,4,20,36,8])
expected_ratio =  np.array([0.1,0.1,0.25,0.45, 0.10])
expected = observed.sum()*expected_ratio
stats.chisquare(observed, expected)

Power_divergenceResult(statistic=4.0, pvalue=0.40600584970983794)

## 5 Chi Squared - Homogenity

If we are testing multiple samles then we are comparing homeogenity. If  it is the same sample but with digfferent attributes e.g. age then we are testing for independance

**Homogenity:**   A high chi square results in a lower p value if the p value is below the significance level then we can conclude that the distributions are different
**Independance:** A high chi square results in a lower p value if the p value is below the significance level then we can conclude that thir is dependance





Age|Alerts|No alerts|Total|
----|---|----|----|
0-29|48|64|112|
    |56|56||
30-59|33|27|60|
    |30|30||
60+|19|9|28|
    |14|14||
Total|100|100|200|

They want to use these results to carry out a \chi^2χ 
2
 \chi, squared test for homogeneity. Assume that all conditions for inference were met.








In [262]:
observed = [[48,33,19], [64,27,9]]
observed
stat, p, dof, expected = stats.chi2_contingency(observed=observed)
print("chisq is", stat) 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Distributions are different (reject H0)')
else:
    print('Distributions are similar (H0 holds true)')

chisq is 6.457142857142857
p value is 0.03961404988497645
Distributions are different (reject H0)


In [263]:
observed = [[6,9,15,6], [78,55,133,58]]
observed
stat, p, dof, expected = stats.chi2_contingency(observed=observed)
print("chisq is", stat) 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Distributions are different (reject H0)')
else:
    print('Distributions are similar (H0 holds true)')

chisq is 1.966296653796654
p value is 0.5794313456996103
Distributions are similar (H0 holds true)


In [269]:
observed = [[203,117,45], [77,36,7], [50,12,3]]
observed
stat, p, dof, expected = stats.chi2_contingency(observed=observed)
print("chisq is", stat) 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Distributions are different (reject H0)')
else:
    print('Distributions are similar (H0 holds true)')
    


chisq is 13.964450883971432
p value is 0.0074093823852925695
Distributions are different (reject H0)


In [270]:
observed = [[30,20], [25,5], [15,5]]
observed
stat, p, dof, expected = stats.chi2_contingency(observed=observed)
print("chisq is", stat) 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Distributions are different (reject H0)')
else:
    print('Distributions are similar (H0 holds true)')

chisq is 5.158730158730159
p value is 0.07582212977799092
Distributions are similar (H0 holds true)


In [272]:
observed = [[6,10,20], [20,13,33], [94,97,307]]
observed
stat, p, dof, expected = stats.chi2_contingency(observed=observed)
print("chisq is", stat) 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Distributions are different (reject H0)')
else:
    print('Distributions are similar (H0 holds true)')

chisq is 6.621232404364934
p value is 0.1573102454361324
Distributions are similar (H0 holds true)


In [273]:
observed = [[38,162], [7,53], [3,37]]
observed
stat, p, dof, expected = stats.chi2_contingency(observed=observed)
print("chisq is", stat) 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Distributions are different (reject H0)')
else:
    print('Distributions are similar (H0 holds true)')

chisq is 4.327876984126984
p value is 0.11487180723651062
Distributions are similar (H0 holds true)


In [278]:
observed = [[28,10,12], [31,4,15]]
observed
stat, p, dof, expected = stats.chi2_contingency(observed=observed)
print("chisq is", stat) 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Distributions are different (reject H0)')
else:
    print('Distributions are similar (H0 holds true)')

chisq is 3.057304277643261
p value is 0.21682772411751125
Distributions are similar (H0 holds true)


In [299]:

m = 76.55
s = 5
x1 = (79-m)/s
x2 = (86.05 -m)/s
norm_cdf1 = stats.norm.cdf(x1)
norm_cdf2 = stats.norm.cdf(x2)
norm_cdf2 - norm_cdf1



0.2833503896013885

In [284]:
norm_cdf2

0.017864420562816542

In [286]:
norm_cdf1- norm_cdf2

0.600047001626136

In [292]:


m = 85
sd = 29  

z = stats.norm.ppf(0.9)
z*sd + m

122.1649954007934